# Running Megadetector on exclosure images
This sections runs megadetector on folders of exclosure images. The images must be renamed in the format *cameraName*\__dd\_mm\_yyyy_\__hh\_mm\_ss_.jpg for the script to work.

In [ ]:
from megadetector.detection.run_detector_batch import load_and_run_detector_batch, write_results_to_file
from megadetector.utils import path_utils
import os
import json
import pandas as pd
import shutil

In [ ]:
image_folder = os.path.expanduser('E:\\Wild deserts photos\\Exclosures\\Thipa\\test')
output_file = os.path.expanduser('E:\\Wild deserts photos\\Exclosures\\Thipa\\test\\results.json')

In [ ]:
image_file_names = path_utils.find_images(image_folder,recursive=True)

In [ ]:
results = load_and_run_detector_batch('MDV5A', image_file_names, checkpoint_frequency = 1000)

In [ ]:
write_results_to_file(results,
                      output_file
                      )

Output file saved at E:\Wild deserts photos\Exclosures\Thipa\test\results.json


{'images': [{'file': 'E:/Wild deserts photos/Exclosures/Thipa/test/testcam1/NSCAM05_01_07_2024_22_34_07.JPG',
   'detections': [{'category': '1',
     'conf': 0.955,
     'bbox': [0.7539, 0.4902, 0.0791, 0.1229]}]},
  {'file': 'E:/Wild deserts photos/Exclosures/Thipa/test/testcam1/NSCAM05_01_07_2024_22_34_08.JPG',
   'detections': [{'category': '1',
     'conf': 0.955,
     'bbox': [0.6303, 0.4993, 0.1118, 0.1354]}]},
  {'file': 'E:/Wild deserts photos/Exclosures/Thipa/test/testcam1/NSCAM05_01_07_2024_22_34_09.JPG',
   'detections': [{'category': '1',
     'conf': 0.96,
     'bbox': [0.2607, 0.5229, 0.2128, 0.1701]}]},
  {'file': 'E:/Wild deserts photos/Exclosures/Thipa/test/testcam1/NSCAM05_01_07_2024_22_34_10.JPG',
   'detections': []},
  {'file': 'E:/Wild deserts photos/Exclosures/Thipa/test/testcam2/NSCAM02_01_08_2024_14_31_44.JPG',
   'detections': [{'category': '1',
     'conf': 0.954,
     'bbox': [0.1293, 0.7999, 0.2802, 0.177]}]},
  {'file': 'E:/Wild deserts photos/Exclosures/

# Processing the results file

In [ ]:
with open(output_file, 'r') as file:
    data = json.load(file)
df = pd.DataFrame(data['images'])

In [45]:
category_map = {'1': 'animal', '2': 'person', '3': 'vehicle'}
confidence_threshold = 0.7
main_dir = 'E:\\Wild deserts photos\\Exclosures\\Thipa\\test\\all_images'
empty_dir = 'E:\\Wild deserts photos\\Exclosures\\Thipa\\test\\all_images\\empty'
os.makedirs(main_dir, exist_ok=True)
os.makedirs(os.path.join(main_dir, empty_dir), exist_ok=True)

In [54]:
for index, row in df.iterrows():
    file_path = row['file']
    detections = row['detections']
    file_name = os.path.basename(file_path)
    valid_categories = set()
    if isinstance(detections, list):
        for detection in detections:
            if isinstance(detection, dict) and detection.get('conf', 0) > confidence_threshold and detection.get('category') in category_map:
                valid_categories.add(category_map[detection['category']])

    source_path = file_path
    destination_base = os.path.join(main_dir) # Keep original subdirectory structure within all_images
    os.makedirs(destination_base, exist_ok=True)
    destination_path = destination_base # Default destination
    if valid_categories:
        # Create a combined folder name if there are multiple categories
        if len(valid_categories) > 1:
            folder_name = '_'.join(sorted(list(valid_categories)))
            destination_path = os.path.join(main_dir, folder_name, file_name)
        elif len(valid_categories) == 1:
            category = valid_categories.pop()
            destination_path = os.path.join(main_dir, category, file_name)

        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        try:
            shutil.move(source_path, destination_path)
            print(f"Moved '{source_path}' to '{destination_path}'")
        except FileNotFoundError:
            print(f"Source file not found: {source_path}")
        except Exception as e:
            print(f"An error occurred while moving '{source_path}': {e}")
    else:
        # Move files with no valid detections to the 'empty' folder
        destination_path = os.path.join(main_dir, empty_dir, file_name)
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        try:
            shutil.move(source_path, destination_path)
            print(f"Moved '{source_path}' to '{destination_path}' (empty)")
        except FileNotFoundError:
            print(f"Source file not found: {source_path}")
        except Exception as e:
            print(f"An error occurred while moving '{source_path}' to empty: {e}")

print("File processing complete!")

Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam1/NSCAM05_01_07_2024_22_34_07.JPG
Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam1/NSCAM05_01_07_2024_22_34_08.JPG
Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam1/NSCAM05_01_07_2024_22_34_09.JPG
Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam1/NSCAM05_01_07_2024_22_34_10.JPG
Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam2/NSCAM02_01_08_2024_14_31_44.JPG
Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam2/NSCAM02_01_08_2024_14_31_45.JPG
Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam2/NSCAM02_01_08_2024_14_31_46.JPG
Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam2/NSCAM02_01_08_2024_16_37_12.JPG
Source file not found: E:/Wild deserts photos/Exclosures/Thipa/test/testcam2/NSCAM02_01_08_2024_16_37_13.JPG
Source file not fou